In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re

C:\Users\paull\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
#connect to top 20  UK cities in tripadvisor and their restaurant links 
URL = 'https://www.tripadvisor.co.uk/Restaurants-g186216-United_Kingdom.html'
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-637ca4cb-0a1f492e332bb59f0bc53de4"}

r = requests.get(URL, headers = headers )

soup = BeautifulSoup(r.content, "html.parser")



In [3]:
#Scrape the links of all the Restaurant pages
places = soup.find_all('div', class_="geo_info")
Cities =[]
for item in places:
    get_text = item.find('a')
    link = get_text.get('href')
    Cities.append(link)
    
print(Cities)
    

['/Restaurants-g186338-London_England.html', '/Restaurants-g186402-Birmingham_West_Midlands_England.html', '/Restaurants-g187069-Manchester_Greater_Manchester_England.html', '/Restaurants-g186525-Edinburgh_Scotland.html', '/Restaurants-g186534-Glasgow_Scotland.html', '/Restaurants-g186411-Leeds_West_Yorkshire_England.html', '/Restaurants-g186337-Liverpool_Merseyside_England.html', '/Restaurants-g186220-Bristol_England.html', '/Restaurants-g186364-Sheffield_South_Yorkshire_England.html', '/Restaurants-g186356-Nottingham_Nottinghamshire_England.html', '/Restaurants-g186334-Leicester_Leicestershire_England.html', '/Restaurants-g186460-Cardiff_South_Wales_Wales.html', '/Restaurants-g186394-Newcastle_upon_Tyne_Tyne_and_Wear_England.html', '/Restaurants-g186273-Brighton_East_Sussex_England.html', '/Restaurants-g186470-Belfast_Northern_Ireland.html', '/Restaurants-g186346-York_North_Yorkshire_England.html', '/Restaurants-g186299-Southampton_Hampshire_England.html', '/Restaurants-g186262-Bourn

In [4]:
#Above links takes you to list of Restaurants in each city, we are interested in VEGAN friendly restaurants
#Simple inspection of html code shows that the "zfz10697" key links to vegan restaurant per city therefore have to update each link

vegan_cities=[]

for link in Cities:
    new_link= "https://www.tripadvisor.co.uk"+ link[:20] + "-zfz10697" + link[20:]
    vegan_cities.append(new_link)
    
print(vegan_cities) #print to test that they are all there

['https://www.tripadvisor.co.uk/Restaurants-g186338-zfz10697-London_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186402-zfz10697-Birmingham_West_Midlands_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g187069-zfz10697-Manchester_Greater_Manchester_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186525-zfz10697-Edinburgh_Scotland.html', 'https://www.tripadvisor.co.uk/Restaurants-g186534-zfz10697-Glasgow_Scotland.html', 'https://www.tripadvisor.co.uk/Restaurants-g186411-zfz10697-Leeds_West_Yorkshire_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186337-zfz10697-Liverpool_Merseyside_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186220-zfz10697-Bristol_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186364-zfz10697-Sheffield_South_Yorkshire_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186356-zfz10697-Nottingham_Nottinghamshire_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186334-zfz10697-Leic

In [5]:
#Now to loop through each city and extract number of search results for vegan option restaurants
#requests loaded the data the page to quickly and the "span" data for search result was empty, and so I have used selenium here
from selenium import webdriver
import time # when in for loop it seemed it loaded info too fast to pick up number of search results, so added a delay
vegan_options=[]
city_names=[]

for link in vegan_cities:
    url= link
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)
    soup= BeautifulSoup(driver.page_source, "html.parser")
    num_search = soup.find("span", class_="SgeRJ").text.strip()[0:4]
    num_vegan= int(num_search)
    vegan_options.append(num_vegan)
    what_city= soup.find("div", class_="easyClear").text.strip()[30:]
    city_names.append(what_city)
 


In [6]:
#save into a dataframe to store data

import pandas as pd

df = pd.DataFrame(list(zip(city_names, vegan_options)), columns=['City','Number of Vegan Friendly Restaurants'])



In [7]:
df

,City,Number of Vegan Friendly Restaurants
0,London,5026
1,Birmingham,546
2,Manchester,517
3,Edinburgh,782
4,Glasgow,616
5,Leeds,547
6,Liverpool,484
7,Bristol,481
8,Sheffield,403
9,Nottingham,311


In [8]:
# import ONS Population_Data
raw_data= pd.read_csv('Population_Data.csv')
raw_data

,Code,Name,Geography,All ages,0,1,2,3,4,5,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,K02000001,UNITED KINGDOM,Country,"67,081,234","701,897","730,219","759,354","783,321","807,539","806,745",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,K03000001,GREAT BRITAIN,Country,"65,185,724","679,949","707,375","735,831","759,033","782,406","781,788",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,K04000001,ENGLAND AND WALES,Country,"59,719,724","631,314","656,313","682,886","704,584","725,691","724,207",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E92000001,ENGLAND,Country,"56,550,138","601,913","625,476","650,226","671,016","690,816","689,190",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E12000001,NORTH EAST,Region,"2,680,763","25,311","26,882","27,877","28,895","29,813","29,512",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Rename Columns to match scraped data
raw_data=raw_data.rename(columns={'Name':'City'})
raw_data=raw_data.rename(columns={'All ages':'Population'})


In [10]:
#Create new data frame for merging

df_pop =raw_data[['City', 'Population']]

df_pop

,City,Population
0,UNITED KINGDOM,"67,081,234"
1,GREAT BRITAIN,"65,185,724"
2,ENGLAND AND WALES,"59,719,724"
3,ENGLAND,"56,550,138"
4,NORTH EAST,"2,680,763"
...,...,...
425,NaN,NaN
426,NaN,NaN
427,NaN,NaN
428,NaN,NaN


In [11]:
#merge datasets
df_merged = pd.merge(df_pop,df, on = "City")
    

In [12]:
df_merged

,City,Population,Number of Vegan Friendly Restaurants
0,Newcastle upon Tyne,"306,824",322
1,Manchester,"555,741",517
2,Liverpool,"500,474",484
3,York,"211,012",277
4,Sheffield,"589,214",403
5,Bradford,"542,128",120
6,Leeds,"798,786",547
7,Leicester,"354,036",282
8,Nottingham,"337,098",311
9,Birmingham,"1,140,525",546


In [13]:
# Find all restaurants per top 20 cities: First get the links

all_cities=[]

for link in Cities:
    new_link= "https://www.tripadvisor.co.uk"+link
    all_cities.append(new_link)
    
print(all_cities)

['https://www.tripadvisor.co.uk/Restaurants-g186338-London_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186402-Birmingham_West_Midlands_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g187069-Manchester_Greater_Manchester_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186525-Edinburgh_Scotland.html', 'https://www.tripadvisor.co.uk/Restaurants-g186534-Glasgow_Scotland.html', 'https://www.tripadvisor.co.uk/Restaurants-g186411-Leeds_West_Yorkshire_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186337-Liverpool_Merseyside_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186220-Bristol_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186364-Sheffield_South_Yorkshire_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186356-Nottingham_Nottinghamshire_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186334-Leicester_Leicestershire_England.html', 'https://www.tripadvisor.co.uk/Restaurants-g186460-Cardiff_Sout

In [14]:
#Scrape the search result data for All restaurants for % calculation

restaurants=[]
for link in all_cities:
    url= link
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)
    soup= BeautifulSoup(driver.page_source, "html.parser")
    num_search = soup.find("span", class_="SgeRJ").text.strip()[0:5]
    #num_rest= int(num_search)
    restaurants.append(num_search)
    
print(restaurants)

['20178', '2044 ', '2019 ', '1934 ', '1888 ', '1683 ', '1599 ', '1262 ', '1253 ', '1018 ', '1052 ', '886 r', '952 r', '1158 ', '822 r', '694 r', '789 r', '614 r', '640 r', '584 r']


In [15]:
restaurants=[20178, 2044 , 2018 , 1934 , 1888 , 1683 , 1599 , 1262 , 1252 , 1018 , 1052 , 886 , 952, 1158, 822 , 694, 789 , 614,640, 584]

In [16]:

df2 = pd.DataFrame(list(zip(city_names, restaurants)), columns=['City','All Restaurants'])


In [17]:
#Merge for complete data frame
df_merged2 = pd.merge(df_merged,df2, on = "City")


In [18]:
#checkpoint
df = df_merged2
df

,City,Population,Number of Vegan Friendly Restaurants,All Restaurants
0,Newcastle upon Tyne,"306,824",322,952
1,Manchester,"555,741",517,2018
2,Liverpool,"500,474",484,1599
3,York,"211,012",277,694
4,Sheffield,"589,214",403,1252
5,Bradford,"542,128",120,640
6,Leeds,"798,786",547,1683
7,Leicester,"354,036",282,1052
8,Nottingham,"337,098",311,1018
9,Birmingham,"1,140,525",546,2044


In [19]:
#Remove commas and convert str to numerics
df = df.replace(',','', regex=True)
df['Population']= pd.to_numeric(df['Population'])

In [20]:
#Calculating Number of Vegan Restaurants per capita

df['Vegan Restaurants per Capita']= (df['Number of Vegan Friendly Restaurants']/ df['Population'])
df['Vegan Restaurants per 100,000']= df['Vegan Restaurants per Capita'] *100000

In [21]:
df['Restaurants per Capita'] = df['All Restaurants']/ df['Population']
df['% of Restaurants with Vegan Option']= (df['Number of Vegan Friendly Restaurants']/df['All Restaurants'])*100

In [22]:
df

,City,Population,Number of Vegan Friendly Restaurants,All Restaurants,Vegan Restaurants per Capita,"Vegan Restaurants per 100,000",Restaurants per Capita,% of Restaurants with Vegan Option
0,Newcastle upon Tyne,306824,322,952,0.001049,104.946158,0.003103,33.823529
1,Manchester,555741,517,2018,0.000930,93.028947,0.003631,25.619425
2,Liverpool,500474,484,1599,0.000967,96.708321,0.003195,30.268918
3,York,211012,277,694,0.001313,131.272155,0.003289,39.913545
4,Sheffield,589214,403,1252,0.000684,68.396202,0.002125,32.188498
5,Bradford,542128,120,640,0.000221,22.134994,0.001181,18.750000
6,Leeds,798786,547,1683,0.000685,68.478917,0.002107,32.501485
7,Leicester,354036,282,1052,0.000797,79.652917,0.002971,26.806084
8,Nottingham,337098,311,1018,0.000923,92.258038,0.003020,30.550098
9,Birmingham,1140525,546,2044,0.000479,47.872690,0.001792,26.712329


In [24]:
#save to csv

df.to_csv('VegRes_Data.csv', index=False)
